## Import packages

In [27]:
import selenium
from selenium import webdriver

import shutil, time, os
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import src
reload(src)

<module 'src' from 'C:\\Users\\Matth\\git\\DataAnalysisWorkbooks\\warcraftLogs\\src.py'>

## Configurations

In [7]:
# Set path variables
path_to_ublock, path_to_download_dir = src.get_path_settings()

In [8]:
# Number of parses to scrape
N_parses = 401
retry_attempts = 1

# Enable/disable SMS
twilio = True

# Enable/disable verbose printouts (debugger)
verbose = True
verbose_rotation = False

## Navigate to WCL

In [9]:
boss = "Eredar Twins"
phase = 1

In [10]:
# Dictionary for browser to navigate to boss page URLs
boss_link_dict = {"High Warlord Naj'entus" : "#boss=601", "Supremus" : "#boss=602", "Shade of Akama" : "#boss=603", 
                  "Teron Gorefiend" : "#boss=604", "Gurtogg Bloodboil" : "#boss=605", "Reliquary of Souls" : "#boss=606", 
                  "Mother Shahraz" : "#boss=607", "The Illidari Council" : "#boss=608", "Illidan Stormrage" : "#boss=609", 
                  "Rage Winterchill" : "#boss=618", "Anetheron" : "#boss=619", "Kaz'rogal" : "#boss=620", 
                  "Azgalor" : "#boss=621", "Archimonde" : "#boss=622",
                  "Brutallus" :  "#boss=725", "Eredar Twins" : "#boss=727"}

In [18]:
boss_page_url, browser = src.load_top_N_scraper(path_to_ublock, boss, boss_link_dict, chrome=True, firefox=False)

## Main code loop

In [32]:
def main():
    
    retry_attempt = 0
    
    # The browser can crash at random due to the page loading too slowly. Rather than restarting the loop manually,
    # use 'retry_attempts' (default = 3) and the following while loop to restart automatically.
    while True: 

        # Twilio is an SMS service that will send a text to your phone when the code finishes/crashes.
        # If you haven't setup twilio then set the flag to False in the Configuration cell.
        if twilio: 
            from twilio.rest import Client
            accountSID, authToken, myTwilioNumber, myCellPhone = src.get_twilio_info()
            twilioCli = Client(accountSID, authToken)

        boss_page_url = f'https://classic.warcraftlogs.com/zone/rankings/1013{boss_link_dict[boss]}&class=Druid&spec=Restoration&metric=hps'
        browser.get(boss_page_url)
        time.sleep(2)

        # First check to see if any of the rankings have changed since last scraping and update the excel file
        print("Checking for rank changes since last scrape...")
        already_recorded_indices = src.get_latest_ranks(browser, boss, boss_link_dict, phase, N_parses)

        print("Rank updates complete.")
        print('-----------')
        time.sleep(2)

        page = 1
        boss_page_url = f'https://classic.warcraftlogs.com/zone/rankings/1013{boss_link_dict[boss]}&class=Druid&spec=Restoration&metric=hps'
        browser.get(boss_page_url)
        time.sleep(2)

        #try:
        print("Beginning data scrape...")

        for i in range(1, N_parses):

            # If the player's parse is already in the spreadsheet, skip to the next parse.
            if i in already_recorded_indices: 
                if i % 100 == 0: 
                    page += 1
                    boss_page_url = f'https://classic.warcraftlogs.com/zone/rankings/1013{boss_link_dict[boss]}&class=Druid&spec=Restoration&metric=hps&page={page}'
                    browser.get(boss_page_url)
                    time.sleep(2) 
                continue

            rank, name, server, region, date, HPS, duration = src.get_boss_data_top_N_scraper(browser, boss, boss_link_dict, i)
            if name in ['Chapu', '世外', '抄能力', '自然帅', 'Tables', 'Genwraek', 'Manamia']: continue  # These players have broken reports, skip

            print(rank, name, server, region)

            link = browser.find_element_by_link_text(name)
            link.click()
            time.sleep(1)

            player_df = pd.DataFrame(pd.np.empty((0, 25)))
            player_df.columns = ["Rank", "Name", "Server", "Date", "Duration", "Haste", "nHealers", "Spriest?", "Innervate?", "Bloodlust?", "Nature's Grace?", "Trinket 1", "Trinket 2", "LB_uptime", "HPS", "% LB (tick) HPS", "% LB (bloom) HPS", "% Rejuv HPS", "% Regrowth HPS", "% Swiftmend HPS", "Rotating on tank?", "Rotation 1", "% Rotation 1", "Rotation 2", "% Rotation 2"]

            if boss in ["Eredar Twins", "M'uru"]:
                src.click_on_element_by_id(browser, "filter-phase-text")
                time.sleep(1)
                src.click_on_element_by_id(browser, f"phase-menu-{phase}")
                time.sleep(1)
            
            temp_url = browser.current_url
            time.sleep(1)

            boss_tanks = src.get_tanks(browser)
            nHealers = src.get_nHealers(browser)

            if boss == "Eredar Twins":
                fire_tank = src.get_fire_tank(browser)
                if fire_tank not in boss_tanks: 
                    boss_tanks.append(fire_tank)

            browser.get(temp_url)
            time.sleep(0.5)

            player_link = browser.find_element_by_link_text(name)
            player_link.click()
            time.sleep(0.5)

            # Scrape HPS data
            LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, LB_uptime = src.get_spell_info(browser, HPS)
            time.sleep(3)

            #Get haste and trinkets
            haste = src.get_haste(browser)
            trinket1, trinket2 = src.get_trinkets(browser)
            time.sleep(1)

            # Check for buffs
            spriest = src.check_spriest(browser)
            innervate, bloodlust, powerInfusion, naturesGrace = src.check_buffs(browser)
            time.sleep(0.5)    

            # Download the cast-sequence CSV.
            src.download_csv(browser, temp_url, "filter-casts-tab", path_to_download_dir, "data/cast_sequence.csv")
            time.sleep(1)

            # Clean the csv
            df = src.clean_cast_sequence_csv()
            df = src.fix_cast_time(df)
            time.sleep(0.5)

            # Get the rotations
            rotation1, rotation1_percent, rotation2, rotation2_percent, rotating_on_tank = src.calculate_rotations(df, boss, boss_tanks, LB_uptime, verbose, verbose_rotation)
            print(f'Rotations: {rotation1} ({rotation1_percent}), {rotation2} ({rotation2_percent})')  

            # Export data and cleanup
            to_append = [rank, name, server + " " + region, date, duration, str(haste), str(nHealers), spriest, innervate, bloodlust, naturesGrace, trinket1, trinket2, LB_uptime, HPS, LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, rotating_on_tank, rotation1, rotation1_percent, rotation2, rotation2_percent]
            src.export_to_excel(boss, phase, to_append, player_df, name, 'top_N_druids', True)

            os.remove(f"data/{boss.replace(' ', '')}_{name}.csv")
            os.remove(f"data/cast_sequence.csv")

            print('-------')
            browser.get(boss_page_url)
            time.sleep(1)

            if i % 100 == 0: 
                page += 1
                boss_page_url = f'https://classic.warcraftlogs.com/zone/rankings/1013{boss_link_dict[boss]}&class=Druid&spec=Restoration&metric=hps&page={page}'
                browser.get(boss_page_url)
                time.sleep(2) 


        print(f"{boss} scraping complete.")
        print("---------------")

        if twilio: message = twilioCli.messages.create(body = 'Data scraping complete!', from_ = myTwilioNumber, to = myCellPhone)
        break

        #except:
        #    retry_attempt += 1
        #    time.sleep(10)
        #    if retry_attempt == retry_attempts: 
        #        if twilio: message = twilioCli.messages.create(body='Data scraping crashed', from_=myTwilioNumber, to=myCellPhone)
        #        break
                
                
if __name__ == "__main__":
    main()

Checking for rank changes since last scrape...
50
100
150
200
250
300
350
400
450
Rank updated: 23 to 24, 灵魂道声, May 14
Rank updated: 24 to 25, 瞌睡丶丶, May 25
Rank updated: 25 to 26, 霸霸不缺德, May 19
Rank updated: 26 to 27, 毛毛姐, May 13
Rank updated: 27 to 28, 牛頭牌沙茶德, May 13
Rank updated: 28 to 29, 后溪沟牛牛, May 19
Rank updated: 29 to 30, 永恒井转圈圈, May 20
Rank updated: 30 to 31, 台式排骨大王, May 23
Rank updated: 31 to 32, 素手绾青丝, May 15
Rank updated: 32 to 33, 氢硫化钠, May 21
Rank updated: 33 to 34, 阿尔忒弥斯丶, May 21
Rank updated: 34 to 35, 暗殇灬, May 24
Rank updated: 35 to 36, Newool, May 22
Rank updated: 36 to 37, 我是小熊, May 22
Rank updated: 37 to 38, 毒奶大王, May 23
Rank updated: 38 to 39, 奶的不能呼吸, May 19
Rank updated: 39 to 40, 全能型人才, May 14
Rank updated: 40 to 41, Kuzrakihing, May 20
Rank updated: 41 to 42, 薺薺, May 20
Rank updated: 42 to 43, Demonkids, May 22
Rank updated: 43 to 44, 照脸一爪, May 19
Rank updated: 44 to 45, 荷醒擎雨时, May 21
Rank updated: 45 to 46, 困了, May 19
Rank updated: 46 to 47, 忠队长, May 16
Rank upd

-------
358 Thazle Blanchy (US)
Lifebloom uptime %: 95.61%
0LB 4I 1RG 0.25
2LB 3I 1RG 0.25
2LB 4I 0RG 0.25
0LB 4I 2RG 0.25
Non-tank rotations %: 0.5
Rotating on tank?: No
Rotations: 0LB 4I 1RG (0.25), 2LB 3I 1RG (0.25)
-------
366 Отбросс Пламегор (EU)
Lifebloom uptime %: 51.05%
0LB 3I 2RG 0.091
1LB 2I 3RG 0.182
0LB 3I 3RG 0.182
0LB 1I 5RG 0.182
0LB 1I 2RG 0.091
2LB 4I 0RG 0.091
1LB 5I 0RG 0.091
1LB 2I 0RG 0.091
Non-tank rotations %: 0.55
Rotating on tank?: No
Rotations: 1LB 2I 3RG (0.182), 0LB 3I 3RG (0.182)
-------
374 Apathý Faerlina (US)
Lifebloom uptime %: 100.00%
2LB 2I 0RG 0.071
2LB 0I 2RG 0.429
2LB 0I 3RG 0.214
2LB 1I 2RG 0.143
2LB 2I 2RG 0.071
1LB 0I 2RG 0.071
Non-tank rotations %: 0
Rotating on tank?: Yes
Rotations: 2LB 0I 2RG (0.429), 2LB 0I 3RG (0.214)
-------
376 香杏 寒脊山小径 (CN)
Lifebloom uptime %: 69.38%
3LB 0I 0RG 0.182
2LB 0I 4RG 0.091
1LB 3I 2RG 0.091
0LB 2I 4RG 0.182
0LB 3I 2RG 0.091
1LB 2I 3RG 0.182
0LB 1I 5RG 0.091
0LB 0I 3RG 0.091
Non-tank rotations %: 0.45
Rotating 

KeyboardInterrupt: 

In [30]:
boss_page_url = f'https://classic.warcraftlogs.com/zone/rankings/1013{boss_link_dict[boss]}&class=Druid&spec=Restoration&metric=hps'
browser.get(boss_page_url)